<a href="https://colab.research.google.com/github/dineshRaja29/Frame-Level-Speech-Recognition/blob/main/EXPERIMENT_II_WITHOUT_OUTPUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = 'green'>Frame-Level Speech Recognition

## <font color = 'green'> EXPERIMENT

* Build LA-DNN
    * A powerful model for speech recognition and good candidate to replace FF-DNN
    * Helps to remove the vanshing gradient problems in deep network
    * https://ieeexplore.ieee.org/document/7472646
* No Augmentation
* No hyper-parameter tuning however used our past knowledge to set hyper-paramters
* Advanced weight initialization, Label smoothing in loss function, gradient clipping and scheduler are used.

In [ ]:
!pip install torch-summary --quiet

In [ ]:
import torch
import numpy as np
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import torchsummary
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

In [ ]:
# ### If you are using colab, you can import google drive to save model checkpoints in a folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir -p /root/.kaggle

In [ ]:
with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"dineshbuswala","key":"a67fefaecacb98180c9d56e1b00b37de"}')

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# commands to download data from kaggle

!kaggle competitions download -c 11785-hw1p2-f23 --force
!mkdir -p '/content/data'

# !unzip -qo /content/11785-hw1p2-f23.zip -d '/content/data'

In [ ]:
!unzip -qo /kaggle/working/11785-hw1p2-f23.zip -d '/content/data'

In [ ]:
# ! rm -r /content/data/11-785-f23-hw1p2/test-clean
# ! rm -r /content/11785-hw1p2-f23.zip

In [ ]:
! ls /content/data/11-785-f23-hw1p2/train-clean-100/mfcc/ | wc -l

In [ ]:
### configuration variables
EPOCHS                          = 4
BATCH_SIZE                      = 2048 * 2
LEFT_CONTEXT                    = 7
RIGHT_CONTEXT       	        = 7
INITIAL_LEARNING_RATE           = 1e-3
L2_PENALTY                      = 1e-5
STEP_SIZE                       = 2
GAMMA                           = 0.1
BASE_DIRECTORY                  = '/content/data/11-785-f23-hw1p2/'
TRAINING_DATA                   = BASE_DIRECTORY + 'train-clean-100'
EVALUATION_DATA                 = BASE_DIRECTORY + 'dev-clean'
PHONEMES                        = ['[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
                                    'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
                                    'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
                                    'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
                                    'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
                                    'V',     'W',     'Y',     'Z',     'ZH'] #,    '[SOS]', '[EOS]']
PHONEMES_TO_INDEX                = {phoneme: idx for idx, phoneme in enumerate(PHONEMES)}
NUMBER_OF_NEURONS                = [2048, 2048, 1024, 1024, 512, 256, 256]
MODEL_DIR                        = "/content"
CLIP_VALUE                       = 1.0
LABEL_SMOOTHING                  = 0.01

In [ ]:
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes = PHONEMES_TO_INDEX,
                 left_context = LEFT_CONTEXT,
                 right_context = RIGHT_CONTEXT):

        self.left_context = left_context
        self.right_context = right_context
        self.phonemes_mapping = phonemes

        self.mfcc_dir = os.path.join(root, 'mfcc')
        self.transcript_dir = os.path.join(root, 'transcript')

        # List and sort mfcc and transcript files
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        transcript_names = sorted(os.listdir(self.transcript_dir))

        # Sanity check
        assert len(mfcc_names) == len(transcript_names), "Mismatch in number of MFCC and transcript files"

        total_frames = 0

        for i in range(len(mfcc_names)):
            mfcc_path = os.path.join(self.mfcc_dir, mfcc_names[i])

            mfcc = np.load(mfcc_path,
                           allow_pickle = False,
                           mmap_mode='r')
            total_frames += mfcc.shape[0]
            del mfcc

        sample_mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[0]), allow_pickle=False)
        self.mfcc_dim = sample_mfcc.shape[1]
        ### Right Padding is added here automatically
        self.mfccs = np.zeros((total_frames + right_context, self.mfcc_dim))
        self.transcripts = [None] * (total_frames + right_context)
        ### Release memory
        del sample_mfcc, total_frames
        gc.collect()

        index = 0
        for i in range(len(mfcc_names)):
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]),
                           allow_pickle = False,
                           mmap_mode = 'r')

            mfcc = (mfcc - mfcc.mean(axis = 1, keepdims = True)) / (mfcc.std(axis = 1, keepdims = True) + 1e-5)

            transcript = np.load(os.path.join(self.transcript_dir, transcript_names[i]),
                                 allow_pickle = False,
                                 mmap_mode='r')[1:-1]

            self.mfccs[index: index + mfcc.shape[0]] = mfcc
            self.transcripts[index: index + len(transcript)] = transcript
            index += mfcc.shape[0]
            del mfcc, transcript

            if i % 1000 == 0:
                gc.collect() ### Release memory

        # Save original dataset length (before adding right padding)
        self.length = len(self.mfccs) - right_context

        # Map transcript phonemes to indices
        self.transcripts = [self.phonemes_mapping.get(p, -1) for p in self.transcripts]

        ### Release memory
        del mfcc_names, transcript_names
        gc.collect()

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        if ind < self.left_context:  # index is less than left context
            # zeros need to prepend with it
            padding = []
            for _ in range(self.left_context - ind):
                padding.append(np.zeros((1, self.mfcc_dim)))

            for i in range(ind):
                padding.append(self.mfccs[i][None, :])

            # Include current and right context frames
            current = self.mfccs[ind][None, :]  # Ensure shape (1, 28)
            right_context = [self.mfccs[i][None, :] for i in range(ind + 1, ind + 1 + self.right_context)]
            frames = np.concatenate(padding + [current] + right_context, axis=0)

        else:  # when index is greater than or equal to left context
            left_context_frames = self.mfccs[ind - self.left_context: ind]
            current = self.mfccs[ind][None, :]  # Ensure shape (1, 28)
            right_context_frames = self.mfccs[ind + 1: ind + 1 + self.right_context]

            frames = np.concatenate([left_context_frames, current, right_context_frames], axis=0)

        frames = frames.flatten()  # Flatten to get 1D data
        frames = torch.FloatTensor(frames)  # Convert to tensor
        phonemes = torch.tensor(self.transcripts[ind], dtype=torch.long)  # Convert label to tensor

        return frames, phonemes


In [ ]:
# Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(TRAINING_DATA)

# Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(EVALUATION_DATA)


In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 2,
    batch_size  = BATCH_SIZE,
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 1,
    batch_size  = BATCH_SIZE,
    pin_memory  = True,
    shuffle     = False
)

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))

In [ ]:
# # Testing code to check if data loaders are working as expecting
# total_batches = len(train_loader)

# for i, (frames, phoneme) in enumerate(train_loader):
#     if i < 10 or i >= total_batches - 10:
#         print(f"Batch {i + 1}/{total_batches}")
#         print("Frames:", frames)
#         print("Phoneme:", phoneme)
#         print("-" * 50)


In [ ]:
class LADNNLayer(torch.nn.Module):
    def __init__(self, in_features, out_features, rank = -1):
        super(LADNNLayer, self).__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.activation = torch.nn.ReLU()
        if rank == -1:
            self.intermediate = in_features // 2
        else:
            self.intermediate = rank
        self.U = torch.nn.Linear(self.in_features, self.intermediate)
        self.V = torch.nn.Linear(self.intermediate, self.out_features)
        self.T = torch.nn.Linear(self.in_features, self.out_features)

        # Apply weight initialization
        self._initialize_weights()

    def _initialize_weights(self):
        print('Initialization of weights using Kaiming')
        for m in [self.U] + [self.V] + [self.T]:
            if isinstance(m, torch.nn.Linear):
                # Kaiming initialization for weights
                torch.nn.init.kaiming_uniform_(m.weight, nonlinearity = 'relu')

    def forward(self, x):
        t_x = self.T(x)
        u_x = self.U(x)
        v_x = self.V(self.activation(u_x))
        return v_x + t_x



class Network(torch.nn.Module):
    def __init__(self, input_size):
        super(Network, self).__init__()

        # Neurons in each layer: input -> hidden(s) -> output
        self.neurons = [input_size] + NUMBER_OF_NEURONS + [len(PHONEMES)]

        layers = []
        for in_features, out_features in zip(self.neurons[:-2], self.neurons[1:-1]):
            layers.append(LADNNLayer(in_features, out_features))

        # Final layer
        layers.append(LADNNLayer(self.neurons[-2], self.neurons[-1]))

        # Combine all into a sequential model
        self.model = torch.nn.Sequential(*layers)

        # Apply weight initialization
        self._initialize_weights()

    def _initialize_weights(self):
        print('Initialization of weights using Kaiming')
        for m in self.model:
            if isinstance(m, torch.nn.Linear):
                # Kaiming initialization for weights
                torch.nn.init.kaiming_uniform_(m.weight, nonlinearity = 'relu')


    def forward(self, x):
        out = self.model(x)
        return out


In [ ]:
INPUT_SIZE = (LEFT_CONTEXT + RIGHT_CONTEXT + 1) * 28
model = Network(INPUT_SIZE).to(device)
# Pass the input size as a tuple, without the batch dimension
torchsummary.summary(model, (INPUT_SIZE,))

In [ ]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing = LABEL_SMOOTHING) # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.Adam(model.parameters(),
                             lr = INITIAL_LEARNING_RATE,
                             weight_decay = L2_PENALTY) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size = STEP_SIZE, gamma = GAMMA
)
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()

        ### Clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_VALUE)

        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]


        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

In [ ]:
best_model_path = os.path.join(MODEL_DIR, "best_model.pt")
best_acc = -np.inf
for epoch in range(EPOCHS):
    ### clean up memory before computation
    torch.cuda.empty_cache()
    gc.collect()


    print(f"\nEpoch {epoch + 1}/{EPOCHS}")

    curr_lr = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = eval(model, val_loader)

    print(f"\tTrain Acc: {train_acc*100:.2f}%\tTrain Loss: {train_loss:.4f}\tLR: {curr_lr:.7f}")
    print(f"\tVal   Acc: {val_acc*100:.2f}%\tVal   Loss: {val_loss:.4f}")

    # Save model at every epoch
    epoch_model_path = os.path.join(MODEL_DIR, f"model_at_epoch_{epoch + 1}.pt")
    torch.save(model.state_dict(), epoch_model_path)

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Updated Best Model at: {best_model_path}")

    ### take step in adjusting the learning rate
    scheduler.step()



In [ ]:

# Load best model after training
model.load_state_dict(torch.load('/content/best_model.pt'))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
model.eval()  # Set model in evaluation mode
predicted = []
groundtruth = []

for frames, phonemes in val_loader:

    # Move data to device
    frames = frames.to(device)
    phonemes = phonemes.to(device)

    # Disable gradient calculation
    with torch.inference_mode():
        logits = model(frames)

    predict = torch.argmax(logits, dim = 1)

    # Detach and move to CPU for evaluation
    predicted.extend(predict.detach().cpu().tolist())
    groundtruth.extend(phonemes.detach().cpu().tolist())

    # Release memory
    del frames, phonemes, logits, predict
    torch.cuda.empty_cache()

# Print classification report
print(classification_report(
    groundtruth,
    predicted,
    target_names = PHONEMES  # Skipping SOS and EOS tokens
))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(groundtruth, predicted)

# Normalize confusion matrix by row (i.e., by true labels)
cm_normalized = cm.astype('float') / cm.sum(axis = 1, keepdims = True)

# Replace NaNs (from division by zero, if any row sum is 0)
cm_normalized = np.nan_to_num(cm_normalized)

# Plot
plt.figure(figsize=(15, 16))
sns.heatmap(cm_normalized,
            annot=True,
            fmt=".1f",
            cmap="Greens",
            xticklabels=PHONEMES,
            yticklabels=PHONEMES,
            cbar_kws={'label': 'Proportion'})

plt.title('Normalized Confusion Matrix', fontsize=16)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()
plt.show()
